## PostgreSQL helpers

In [1]:
import psycopg2
conn = psycopg2.connect("dbname=geonames user=deserg")

cursor = conn.cursor()

In [4]:
Levels = ['cn', 'rn', 'ppl']
CityLevel = 2
RegionLevel = 1
CountryLevel = 0

find_precisely_command = """select *, b.name from geo_{0} a inner join geo_{0}_names b
    on a.geonameid = b.geonameid
    where b.lower_name = '{1}';"""

find_with_errs_command = """select *, b.lower_name <-> '{1}' AS dist
    from geo_{0} a
    inner join geo_{0}_names b
    on a.geonameid = b.geonameid
    order by dist
    limit 10;"""

find_names_command = """select name from geo_{0}_names where geonameid = '{1}';"""

find_id_precisely_command = """select * from geo_{0} where geonameid = {1};"""

# Place: 'geonameid', 'name', 'class', 'code', 'country', 'region'
def place_from_record(record):
    place = {}
    place['geonameid'] = record[0]
    place['name'] = record[8]
    place['class'] = record[1]
    place['code'] = record[2]
    place['country'] = record[3]
    place['region'] = record[4]
    place['pop'] = record[5]
    return place

def find_ids_precisely(geonameid, level_ind):
    if level_ind < 0 or level_ind >= len(Levels):
        print('bad ind')
        return []
    try:
        cursor.execute(find_id_precisely_command.format(Levels[level_ind], geonameid))
    except Exception as ex:
        print(ex)
        self.conn.rollback()
        return []
    conn.commit()
    tup = self.cursor.fetchone()
    if tup:
        return tup
    else:
        return ()

def find_records_precisely(location, level_ind):
    if level_ind < 0 or level_ind >= len(Levels):
        return []
    cursor.execute(find_precisely_command.format(Levels[level_ind], location))
    list_tup = cursor.fetchall()
    results = []
    for tup in list_tup:
        results.append(place_from_record(tup))
    return results

def find_records_with_errs(location, level_ind):
    if level_ind < 0 or level_ind >= len(Levels):
        return []
    cursor.execute(find_with_errs_command.format(Levels[level_ind], location))
    list_tup = cursor.fetchall()
    results = []
    for tup in list_tup:
        results.append(place_from_record(tup))
    return results

def find_names(geonameid, level_ind):
    if level_ind < 0 or level_ind >= len(levels):
        return []
    cursor.execute(find_names_command.format(levels[level_ind], geonameid))
    list_tup = cursor.fetchall()
    results = []
    for tup in list_tup:
        results.append(tup[0])
    return results

In [10]:
find_records_with_errs('saint petersburg', 2)

#cursor.execute(find_with_errs_command.format(levels[CityLevel], 'Sao Paulo'))
print(cursor.fetchone())
#cursor.execute(find_id_precisely_command.format(Levels[1], '4829764'))

None


In [174]:
with open('')

[{'class': 'P',
  'code': 'PPLC',
  'country': 'RU',
  'geonameid': 524901,
  'name': 'Mosco',
  'region': '48'},
 {'class': 'P',
  'code': 'PPL',
  'country': 'US',
  'geonameid': 4092267,
  'name': 'Moscow',
  'region': 'AL'},
 {'class': 'P',
  'code': 'PPL',
  'country': 'US',
  'geonameid': 4393217,
  'name': 'Moscow',
  'region': 'MO'},
 {'class': 'P',
  'code': 'PPLC',
  'country': 'RU',
  'geonameid': 524901,
  'name': 'Moscoƿ',
  'region': '48'},
 {'class': 'P',
  'code': 'PPL',
  'country': 'US',
  'geonameid': 4399153,
  'name': 'Moscow',
  'region': 'MO'},
 {'class': 'P',
  'code': 'PPLC',
  'country': 'RU',
  'geonameid': 524901,
  'name': 'Moscou',
  'region': '48'},
 {'class': 'P',
  'code': 'PPLC',
  'country': 'RU',
  'geonameid': 524901,
  'name': 'Moscow',
  'region': '48'},
 {'class': 'P',
  'code': 'PPLA2',
  'country': 'US',
  'geonameid': 5601538,
  'name': 'Moscow',
  'region': 'ID'},
 {'class': 'P',
  'code': 'PPL',
  'country': 'US',
  'geonameid': 5202009,
  '

## User locations

In [182]:
df = pd.read_csv('geo/toponims-utf8.txt', '\t')#[['author_name', 'place', 'region']]
df[:3]

,author_name,place_num,place,region,country,Unnamed: 5
0,KenigsberginMoscow,1,Московская область,NaN,NaN,NaN
1,Куйбышев-курортнаводах_Москва,1,Москва,NaN,NaN,NaN
2,Balkan,1,NaN,NaN,NaN,NaN


In [185]:
df.dtypes
failed = pd.DataFrame(columns = df.columns)
failed.dtypes

author_name    object
place_num      object
place          object
region         object
country        object
Unnamed: 5     object
dtype: object

In [179]:
toponims = pd.DataFrame(columns=['author_name', 'geonameid', 'level_ind'])
failed_toponims = pd.DataFrame()

for _, record in df.iterrows():
    author_place = record[['author_name']]
    place = record[['place']]
    region = record[['region']]
    
    cities_from_pl = find_records_precisely(place, CityLevel)
    if cities_from_pl:
        
        city_ = cities_from_pl[0]
        toponims.insert(author_place, city_['geonameid'], CityLevel)
        continue
    
    
    
    regions_from_pl = find_records_precisely(place, RegionLevel)
    if regions_from_pl:
        region_ = regions_from_pl[0]
        toponims.insert(author_place, region_['geonameid'], RegionLevel)
        continue

    regions_from_rg = find_records_precisely(region, RegionLevel)
    if regions_from_rg:
        region_ = regions_from_rg[0]
        toponims.insert(author_place, region_['geonameid'], RegionLevel)
        continue
    
    country_from_rg = find_records_precisely(region, CountryLevel)
    if country_from_rg:
        country_ = country_from_rg[0]
        toponims.insert(author_place, country_['geonameid'], CountryLevel)
        continue
    
    print('failed with', author_place)
    failed_toponims.insert(record)

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [202]:
failed = 'user_locs/failed'

toponims = set()
with open(failed) as inp:
    for line in inp:

SyntaxError: unexpected EOF while parsing (<ipython-input-202-8d9e3b10e416>, line 6)

In [ ]:
pd.read_csv()